In [1]:
suppressPackageStartupMessages({
  library(Seurat)
  library(monocle3)
  library(SeuratWrappers)
  library(dplyr)
  library(Matrix)
  library(ggplot2)
  library(patchwork)
  library(SingleCellExperiment)
})

Warning message:
“package ‘Seurat’ was built under R version 4.4.3”
Warning message:
“package ‘SeuratObject’ was built under R version 4.4.3”
Warning message:
“package ‘sp’ was built under R version 4.4.3”
Warning message:
“package ‘matrixStats’ was built under R version 4.4.3”
Warning message:
“package ‘dplyr’ was built under R version 4.4.3”
Warning message:
“package ‘Matrix’ was built under R version 4.4.3”
Warning message:
“package ‘patchwork’ was built under R version 4.4.3”


In [ ]:
adult_obj <- readRDS("/mnt/18T/chibao/gliomas/data/upstream/scRNA/official/integrated_v5_optimized/adult/harmony_cleaned_annotated_v2.rds")
adult_obj

In [ ]:
table(adult_obj$general_cell_type)

malignant_obj <- subset(
  adult_obj,
  subset = general_cell_type == "Malignant"
)

dim(malignant_obj)

In [ ]:
DefaultAssay(malignant_obj) <- "SCT"  # critical for as.cell_data_set()

In [ ]:
cds <- as.cell_data_set(malignant_obj)

cds

In [ ]:
head(colnames(cds))
head(rownames(cds))

colData(cds) |> as.data.frame() |> head()
rowData(cds) |> as.data.frame() |> head()

In [ ]:
if (is.null(rowData(cds)$gene_short_name)) {
  rowData(cds)$gene_short_name <- rownames(cds)
}

In [ ]:
# Ensure the same cells (order) in Seurat and CDS
all(colnames(malignant_obj) == colnames(cds))
# If FALSE, reorder:

cds <- cds[, colnames(malignant_obj)]

In [ ]:
# Extract Harmony embeddings
harm <- Embeddings(malignant_obj, "harmony")  # matrix: cells x dims
dim(harm)
head(harm[, 1:5])

In [ ]:
num_dim <- min(50, ncol(harm))
harm_use <- harm[, 1:num_dim]

In [ ]:
cds <- preprocess_cds(
  cds,
  num_dim = num_dim,                # same as Harmony
  method = "PCA",
  norm_method = "log"               # fine for SCT
)

In [ ]:
reducedDims(cds)$PCA <- harm_use

In [ ]:
umap_harm <- Embeddings(malignant_obj, "umap.harmony")
reducedDims(cds)$UMAP <- umap_harm

In [ ]:
# cds <- reduce_dimension(
#   cds,
#   reduction_method   = "UMAP",
#   preprocess_method  = "PCA",
#   umap.metric        = "cosine",
#   umap.min_dist      = 0.1,
#   umap.n_neighbors   = 30L
# )

In [ ]:
cds <- cluster_cells(
  cds,
  reduction_method = "UMAP",
  k = 20          # adjust; smaller = fewer, bigger = more clusters
)

table(partitions(cds))
table(clusters(cds))

In [ ]:
cds <- learn_graph(
  cds,
  use_partition = TRUE,     # or FALSE if you want one global trajectory
  close_loop = FALSE        # TRUE if you suspect cycles
)

In [ ]:
plot_cells(
  cds,
  color_cells_by = "seurat_clusters",  # or any meta column
  label_groups_by_cluster = TRUE,
  label_leaves = TRUE,
  label_branch_points = TRUE
)

In [ ]:
get_earliest_principal_node <- function(cds, time_bin="OPC_like"){
  cell_ids <- which(colData(cds)[, "general_cell_type"] == time_bin)

  closest_vertex <-
    cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])

  root_pr_nodes <-
    igraph::V(principal_graph(cds)[["UMAP"]])$name[
      as.numeric(names(which.max(table(closest_vertex[cell_ids, ]))))
    ]

  root_pr_nodes
}

cds <- order_cells(cds, root_pr_nodes = get_earliest_principal_node(cds))


In [ ]:
# table(colData(cds)$general_cell_type)

# root_cells <- colnames(cds)[cds$general_cell_type == "OPC_like"]

# length(root_cells)

# cds <- order_cells(
#   cds,
#   root_cells = root_cells
# )

In [ ]:
pseudo <- pseudotime(cds)
summary(pseudo)

# Example: show distribution
hist(pseudo, breaks = 50, main = "Pseudotime distribution", xlab = "Monocle3 pseudotime")

In [ ]:
plot_cells(
  cds,
  color_cells_by = "pseudotime",
  label_cell_groups = FALSE,
  label_leaves = TRUE,
  label_branch_points = TRUE
)

In [ ]:
p1 <- plot_cells(
  cds,
  color_cells_by = "pseudotime",
  group_cells_by = "general_cell_type",
  label_cell_groups = TRUE,
  label_leaves = FALSE,
  label_branch_points = FALSE
)

p2 <- plot_cells(
  cds,
  color_cells_by = "general_cell_type",
  label_cell_groups = TRUE,
  label_leaves = FALSE,
  label_branch_points = FALSE
)

p1 + p2